In [69]:
%%capture
%pip install pandas matplotlib

In [28]:
import time
import requests
import pandas as pd
import matplotlib.pyplot as plt

URL = "http://192.168.0.249"
r = requests.get(f"{URL}/wavelength")
nm = r.json()["nm"]

In [ ]:
start = time.time()
readings = []
intusec = 5000
intstep = 500
valmin, valmax = 1900, 2100
while time.time() - start < 60*60*2:
    tries = 0
    retry = True
    while tries < 10 and retry:
        r = requests.get(f"{URL}/spectrum/{intusec}")
        data = r.json()
        y = data["spec"]
        maxval = max(y)
        if maxval < valmin:
            intusec += intstep
            retry = True
        elif maxval > valmax:
            intusec -= intstep
            retry = True
        else:
            retry = False
        tries += 1
        print(".", end="", flush=True)
        
    print(f"ts={data['ts']}, readtimes={data['readtimes']}, maxval={maxval}, {tries=}")
    readings.append(data)
    time.sleep(60)

In [ ]:
df = pd.json_normalize(readings)
df["time"] = pd.to_datetime(df.ts, unit="s") - datetime.timedelta(hours=7)
df.head()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,4))
ax.set_title("Spectrum")
ax.set_xlabel("Wavelength (nm)")
ax.set_ylabel("Relative spectrum")
#ax.set_ylim([0, 3.5])
ax.set_xlim([300, 890])
for i in range(1, len(df), 1):
    y = [x / df.loc[i, "integration"] for x in df.loc[i, "spec"]]
    ax.plot(nm, y, label=df.loc[i, "time"])
#ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,4))
ax.set_title("Spectrum")
ax.set_xlabel("Wavelength (nm)")
ax.set_ylabel("Relative spectrum")
#ax.set_ylim([0, 3.5])
ax.set_xlim([300, 890])
ref = df.loc[0, "spec"]
for i in range(1, len(df), 10):
    y = [r - x for r,x in zip(ref, df.loc[i, "spec"])]
    ax.plot(nm, y, label=df.loc[i, "time"])
#ax.legend()
plt.show()